In [1]:
import os
import json
import requests

from dotenv import load_dotenv
from typing import List

from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langgraph.prebuilt import create_react_agent

In [2]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROK_API_KEY")
SECTORS_API_KEY = os.getenv("SECTOR_API_KEY")

## General Retrival From Sector

In [3]:
def retrieve_from_endpoint(url: str) -> dict:
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)

## Tool

In [4]:
@tool
def get_company_overview(stock:str)-> str:
    """
    Get company overview from sector API
    """

    url = f"https://api.sectors.app/v1/company/report/{stock}/?sections=overview"
    
    return retrieve_from_endpoint(url)

In [5]:
import datetime
@tool
def get_today_date()->str:
    """
    Get today's date
    """
    return datetime.datetime.now().date().strftime("%Y-%m-%d")

In [6]:
@tool
def get_top_companies_by_tx_volume(start_date: str, end_date: str, top_n: int = 5) -> str:
    """
    Get top companies by transaction volume
    """
    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"

    return retrieve_from_endpoint(url)

In [7]:
@tool
def get_daily_tx(stock: str, start_date: str, end_date: str) -> str:
    """
    Get daily transaction for a stock
    """
    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"

    return retrieve_from_endpoint(url)

In [8]:
tools = [
    get_company_overview,
    get_top_companies_by_tx_volume,
    get_daily_tx,
    get_today_date
]

## Create LLM

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""Answer the following queries factually. Whenever you return a 
            list of names, return also the corresponding values for each name. 
            Always answer in markdown table if you can.
            """,
        ),
        ("human", "{text}"),
        MessagesPlaceholder("agent_scratchpad"),
        
    ]
)

llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

## Agent Creation

In [10]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

query_1 = "What are the top 3 companies by transaction volume over the last 7 days?"
query_2 = "Based on the closing prices of BBCA between 1st and 30th of June 2024, are we seeing an uptrend or downtrend? Try to explain why."
query_3 = "What is the company with the largest market cap between BBCA and BREN? For said company, retrieve the email, phone number, listing date and website for further research."
query_4 = "What is the performance of GOTO (symbol: GOTO) since its IPO listing?"
query_5 = "If i had invested into GOTO vs BREN on their respective IPO listing date, which one would have given me a better return over a 90 day horizon?"

queries = [query_1, query_2, query_3, query_4, query_5]

for query in queries:
    print("Question:", query)
    result = agent_executor.invoke({"text": query})
    print("Answer:", "\n", result["output"], "\n\n======\n\n")

Question: What are the top 3 companies by transaction volume over the last 7 days?
Answer: 
 The top 3 companies by transaction volume over the last 7 days are as follows:

| Date | Company Name | Price | Symbol | Volume |
| --- | --- | --- | --- | --- |
| 2023-09-25 | PT GoTo Gojek Tokopedia Tbk | 87 | GOTO.JK | 1603642100 |
| 2023-09-25 | Darma Henwa Tbk | 77 | DEWA.JK | 1464087000 |
| 2023-09-25 | PT Era Media Sejahtera Tbk | 73 | DOOH.JK | 915160700 |
| 2023-09-26 | PT GoTo Gojek Tokopedia Tbk | 87 | GOTO.JK | 5154826100 |
| 2023-09-26 | Darma Henwa Tbk | 71 | DEWA.JK | 1821746400 |
| 2023-09-26 | PT Era Media Sejahtera Tbk | 98 | DOOH.JK | 814239400 |
| 2023-09-27 | PT GoTo Gojek Tokopedia Tbk | 86 | GOTO.JK | 2659270500 |
| 2023-09-27 | Nusantara Infrastructure Tbk | 224 | META.JK | 992850400 |
| 2023-09-27 | Darma Henwa Tbk | 79 | DEWA.JK | 708982200 |
| 2023-09-29 | PT GoTo Gojek Tokopedia Tbk | 85 | GOTO.JK | 2859965800 |
| 2023-09-29 | Darma Henwa Tbk | 85 | DEWA.JK | 9911228

In [14]:
from langgraph.checkpoint.memory import MemorySaver
from langchain.schema import HumanMessage


memory = MemorySaver()

In [15]:
system_message = "You are an expert tool calling agent meant for financial data retriever and summarization. Use tools to get the information you need. If you do not know the answer to a question, say so."
app = create_react_agent(llm, tools,state_modifier=system_message,checkpointer=memory)
out = app.invoke({
        "messages": [
            HumanMessage(
                content="Give me an overview of ADRO"
            )
        ]
    }, config={"configurable": {"thread_id": "supertype"}})


# result:
print(out["messages"][-1].content)

Adaro Energy Indonesia Tbk, listed as ADRO.JK, is a coal production company operating in the energy sector. It is headquartered at Menara Karya 23rd Floor Jl. H.R. Rasuna Said, Block X-5, Kav. 1-2 Jakarta 12950. The company has a market capitalization of 124,265,147,924,480, ranking it 17th in the market. It has 13,180 employees and is listed on the Main board of the stock exchange since July 16, 2008. The latest closing price was 4,040, with a daily close change of 0.0279898218829517. You can contact them via email at mahardika.putranto@adaro.com or aditya.sudjono@adaro.com, or by phone at 021-25533000. Their website is www.adaro.com.


In [ ]:
system_message = "You are an expert tool calling agent meant for financial data retriever and summarization. Use tools to get the information you need. If you do not know the answer to a question, say so. Whenever possible, answer with markdown-formatted code."
app = create_react_agent(llm, tools, state_modifier=system_message)


In [ ]:
from langchain.schema import HumanMessage 
def query_app(text: str) -> str:
    out = app.invoke(
        {
            "messages": [
                HumanMessage(text),
            ]
        }
    )
    # return out["messages"][-1].content
    return out["messages"]

In [ ]:
out_agent = query_app("Give me an overview of BBRI")
print(out_agent)

[HumanMessage(content='Give me an overview of BBRI', additional_kwargs={}, response_metadata={}, id='80fce9bd-7d4d-44e9-9b67-a3be6280d786'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9111', 'function': {'arguments': '{"stock": "BBRI"}', 'name': 'get_company_overview'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 506, 'total_tokens': 535, 'completion_time': 0.089022853, 'prompt_time': 0.040643959, 'queue_time': 0.000932121000000001, 'total_time': 0.129666812}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-23042742-857d-4d3d-8091-5b3bf402ed39-0', tool_calls=[{'name': 'get_company_overview', 'args': {'stock': 'BBRI'}, 'id': 'call_9111', 'type': 'tool_call'}], usage_metadata={'input_tokens': 506, 'output_tokens': 29, 'total_tokens': 535}), ToolMessage(content='{"symbol": "BBRI.JK", "company_name": "PT Ba

In [ ]:
print(out_agent[-1].content)

The company you're interested in is PT Bank Rakyat Indonesia (Persero) Tbk, listed under the ticker symbol BBRI.JK. It operates in the Financials sector, specifically in the Banks sub-industry. The company is headquartered at Gedung BRI I Lantai 20, Jl. Jenderal Sudirman Kav.44-46, Jakarta Pusat 10210. They have a total of 80,257 employees and are listed on the Main board of the stock exchange since November 10, 2003. Their latest closing price was 4,720, with a market capitalization of 712,569,769,689,088, ranking them 4th in the market capitalization. For more information, you can visit their website at www.bri.co.id or contact them via email at humas@bri.co.id; corsec@bri.co.id or phone at 021 - 575 1966.


In [ ]:
for chunk in app.stream(
    {
        "messages": [
            HumanMessage(
                content="Top 5 companies with the highest market cap, along with their full company name and market cap"
            )
        ]
    }
):
    print(chunk,flush=True)
    print("# ----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2tzk', 'function': {'arguments': '{"top_n": 5}', 'name': 'get_top_companies_by_market_cap'}, 'type': 'function'}, {'id': 'call_ckpf', 'function': {'arguments': '{"top_n": 5}', 'name': 'get_top_companies_by_market_cap'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 518, 'total_tokens': 582, 'completion_time': 0.20050161, 'prompt_time': 0.038503865, 'queue_time': 0.0010611650000000028, 'total_time': 0.239005475}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5a822a66-5e61-4276-aca2-f0ff9db30b62-0', tool_calls=[{'name': 'get_top_companies_by_market_cap', 'args': {'top_n': 5}, 'id': 'call_2tzk', 'type': 'tool_call'}, {'name': 'get_top_companies_by_market_cap', 'args': {'top_n': 5}, 'id': 'call_ckpf', 'type': 'tool_call'}], usage_metadata={'